<a href="https://colab.research.google.com/github/ClaudeJang/Duran-Duran-Returns/blob/master/%EB%8D%B0%EC%9D%B4%EC%BD%98%ED%8C%8C%EC%9D%BC%EC%A7%84%ED%96%89%EC%A4%91(%EC%A0%95%ED%98%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools

!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null

!apt-get update -qq 2>&1 > /dev/null

!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth

auth.authenticate_user()

from oauth2client.client import GoogleCredentials

creds = GoogleCredentials.get_application_default()

import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
# from google.colab import auth

# auth.authenticate_user()

In [0]:
!mkdir -p drive

!google-drive-ocamlfuse drive

In [3]:
cd drive

/content/drive


In [0]:
# ls -ltr

total 0


In [4]:
import pandas as pd
import numpy as np
import warnings 
import os # 운영체제에서 제공되는 기능을 파이썬에서 수행 
import keras


Using TensorFlow backend.


전처리 다시 진행


In [0]:
train = pd.read_csv('./Colab Notebooks/data/train_tuned.csv', encoding = 'utf-8')
train_original = pd.read_csv('./Colab Notebooks/data/train.csv', encoding = 'utf-8')

In [17]:
set(train_original.transaction_date)
#  1-10 : 월 초, 11-20 : 월 중, 21-28 : 월 말 
train.head()
train['new_date'] = train_original['transaction_date']
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601458 entries, 0 to 1601457
Data columns (total 38 columns):
apartment_id                          1601458 non-null int64
Seoul                                 1601458 non-null int64
Busan                                 1601458 non-null int64
transaction_date                      1601458 non-null int64
year_of_completion                    1601458 non-null int64
latitude                              1601458 non-null float64
longitude                             1601458 non-null float64
central                               1601458 non-null int64
district                              1601458 non-null int64
individual                            1601458 non-null int64
cogeneration                          1601458 non-null int64
gas                                   1601458 non-null int64
corridor                              1601458 non-null int64
mixed                                 1601458 non-null int64
stairway                     

In [19]:
train.head()

,apartment_id,Seoul,Busan,transaction_date,year_of_completion,latitude,longitude,central,district,individual,...,tallest_building_in_sites,lowest_building_in_sites,total_household_count_in_sites,total_household_count_of_area_type,total_parking_capacity_in_site,household_loan,mean_loan,mortgage loan,transaction_real_price,new_date
0,5584,0,1,20,1999,37.585965,127.000231,0,0,1,...,0.040000,0.043478,0.005469,0.015541,0.016691,0.568465,0.561404,0.605691,215000000.0,11~20
1,5584,0,1,20,1999,37.585965,127.000231,0,0,1,...,0.040000,0.043478,0.005469,0.003378,0.016691,0.568465,0.561404,0.605691,200000000.0,11~20
2,5059,0,1,20,1992,37.580511,127.014016,0,0,1,...,0.120000,0.115942,0.071840,0.067905,0.092361,0.568465,0.561404,0.605691,168000000.0,11~20
3,2816,0,1,20,1993,37.580324,127.011788,0,0,1,...,0.133333,0.144928,0.121212,0.095946,0.092361,0.568465,0.561404,0.605691,165000000.0,11~20
4,2816,0,1,20,1993,37.580324,127.011788,0,0,1,...,0.133333,0.144928,0.121212,0.037838,0.092361,0.568465,0.561404,0.605691,280000000.0,11~20


In [0]:
list = [] 
for i in range(len(train.new_date)):
    if train.new_date[i] == '1~10':
        list.append('월초')
    elif train.new_date[i] =='11~20':
        list.append('월중')
    else:
        list.append('월말')
        

In [0]:
# set(train.year_of_completion)
list2 = []
for i in range(len(train.year_of_completion)): 
    if train.year_of_completion[i] < 1979:
        list2.append('1세대')
    elif train.year_of_completion[i] < 1990:
        list2.append('2세대')
    elif train.year_of_completion[i] < 2000:
        list2.append('3세대')
    elif train.year_of_completion[i] < 2010:
        list2.append('4세대')
    else:
        list2.append('5세대')
        

In [0]:
list3 = []
for i in range(len(train.transaction_month)):
    if train.transaction_month[i] < 4 and train.transaction_month[i] >=1:
        list3.append('q1')
    elif train.transaction_month[i] < 7 and train.transaction_month[i] >=4:
        list3.append('q2')
    elif train.transaction_month[i] < 10 and train.transaction_month[i] >=7:
        list3.append('q3')
    else:
        list3.append('q4')

In [0]:
train['citycode'] = train.city1.astype(str).str.cat(train.city2.astype(str))
# train2.transaction_year.astype(str) # 거래년도 

In [0]:
list4 = []
s_center = ['11110','11140','11170']
s_eastnorth = ['11200','11215','11230','11260','11290','11305','11320','11350'] 
s_westnorth = ['11380','11410','11440'] 
s_westsouth = ['11470','11500','11530','11545','11560','11590','11620']
s_eastsouth = ['11650','11680','11710','11740']
p_center = ['26110','26140','26170','26200','26230','26290','26470','26500']
p_east = ['26350','26410','26260','26710']
p_west=['26320','26440','26530','26380']

for i in range(len(train.citycode)):
    if train.citycode[i] in s_center:
        list4.append('도심권')
    elif train.citycode[i] in s_eastnorth:
        list4.append('동북권')
    elif train.citycode[i] in s_westnorth:
        list4.append('서북권')
    elif train.citycode[i] in s_westsouth:
        list4.append('서남권')
    elif train.citycode[i] in s_eastsouth:
        list4.append('동남권')
    elif train.citycode[i] in p_center:
        list4.append('중부산권')
    elif train.citycode[i] in p_east:
        list4.append('동부산권')
    else:
        list4.append('서부산권')
        

In [0]:
train['district_seoul_pusan'] = list4 
train['transaction_quarter'] = list3
train['APT_generation'] = list2
train['new_date2'] = list

In [0]:
train['transaction'] = train.transaction_year.astype(str).str.cat(train.transaction_quarter)

In [35]:
# train.info()
# train2 = train.drop(columns = ['transaction_date','year_of_completion','Seoul','Busan','latitude','longitude','city1','city2','city3','city4','household_loan','mean_loan','room_id','central','district','individual','cogeneration','gas','corridor','mixed','stairway','transaction_year','transaction_month','new_date','citycode','transaction_quarter'])
train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601458 entries, 0 to 1601457
Data columns (total 18 columns):
apartment_id                          1601458 non-null int64
apartment_building_count_in_sites     1601458 non-null float64
bathroom_count                        1601458 non-null float64
exclusive_use_area                    1601458 non-null float64
floor                                 1601458 non-null float64
room_count                            1601458 non-null float64
supply_area                           1601458 non-null float64
tallest_building_in_sites             1601458 non-null float64
lowest_building_in_sites              1601458 non-null float64
total_household_count_in_sites        1601458 non-null float64
total_household_count_of_area_type    1601458 non-null float64
total_parking_capacity_in_site        1601458 non-null float64
mortgage loan                         1601458 non-null float64
transaction_real_price                1601458 non-null float64
distric

In [0]:
train2.to_csv('./Colab Notebooks/data/final0120.csv',encoding = 'utf-8')

In [37]:
train0120 = pd.read_csv('./Colab Notebooks/data/final0120.csv', encoding='utf-8', index_col = 0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [38]:
train0120.head()

,apartment_id,apartment_building_count_in_sites,bathroom_count,exclusive_use_area,floor,room_count,supply_area,tallest_building_in_sites,lowest_building_in_sites,total_household_count_in_sites,total_household_count_of_area_type,total_parking_capacity_in_site,mortgage loan,transaction_real_price,district_seoul_pusan,APT_generation,new_date2,transaction
0,5584,0.00000,0.2,0.111218,0.119048,0.125,0.112894,0.040000,0.043478,0.005469,0.015541,0.016691,0.605691,215000000.0,도심권,3세대,월중,2006q1
1,5584,0.00000,0.2,0.100252,0.142857,0.250,0.102380,0.040000,0.043478,0.005469,0.003378,0.016691,0.605691,200000000.0,도심권,3세대,월중,2006q1
2,5059,0.03252,0.2,0.137272,0.142857,0.250,0.129581,0.120000,0.115942,0.071840,0.067905,0.092361,0.605691,168000000.0,도심권,3세대,월중,2006q1
3,2816,0.04878,0.2,0.172966,0.178571,0.250,0.166625,0.133333,0.144928,0.121212,0.095946,0.092361,0.605691,165000000.0,도심권,3세대,월중,2006q1
4,2816,0.04878,0.4,0.323339,0.130952,0.500,0.266898,0.133333,0.144928,0.121212,0.037838,0.092361,0.605691,280000000.0,도심권,3세대,월중,2006q1


In [0]:
# train2.head()
train3 = pd.get_dummies(train0120, prefix = ['new_date2','APT_generation','district_seoul_pusan','transaction'])


In [50]:
train3.head()

,apartment_id,apartment_building_count_in_sites,bathroom_count,exclusive_use_area,floor,room_count,supply_area,tallest_building_in_sites,lowest_building_in_sites,total_household_count_in_sites,...,transaction_2016q3,transaction_2016q4,transaction_2017q1,transaction_2017q2,transaction_2017q3,transaction_2017q4,transaction_2018q1,transaction_2018q2,transaction_2018q3,transaction_2018q4
0,5584,0.00000,0.2,0.111218,0.119048,0.125,0.112894,0.040000,0.043478,0.005469,...,0,0,0,0,0,0,0,0,0,0
1,5584,0.00000,0.2,0.100252,0.142857,0.250,0.102380,0.040000,0.043478,0.005469,...,0,0,0,0,0,0,0,0,0,0
2,5059,0.03252,0.2,0.137272,0.142857,0.250,0.129581,0.120000,0.115942,0.071840,...,0,0,0,0,0,0,0,0,0,0
3,2816,0.04878,0.2,0.172966,0.178571,0.250,0.166625,0.133333,0.144928,0.121212,...,0,0,0,0,0,0,0,0,0,0
4,2816,0.04878,0.4,0.323339,0.130952,0.500,0.266898,0.133333,0.144928,0.121212,...,0,0,0,0,0,0,0,0,0,0


In [0]:
train3.to_csv('./Colab Notebooks/data/traindatakeras.csv',encoding = 'utf-8')
test.to_csv('./Colab Notebooks/data/testdatakeras.csv', encoding = 'utf-8')

In [128]:
# test10 = pd.read_csv('./Colab Notebooks/data/testdatakeras.csv',encoding = 'utf-8',index_col=0)
test10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3918 entries, 0 to 3917
Data columns (total 48 columns):
apartment_id                          3918 non-null int64
apartment_building_count_in_sites     3918 non-null float64
bathroom_count                        3918 non-null float64
exclusive_use_area                    3918 non-null float64
floor                                 3918 non-null float64
room_count                            3918 non-null float64
supply_area                           3918 non-null float64
tallest_building_in_sites             3918 non-null float64
lowest_building_in_sites              3918 non-null float64
total_household_count_in_sites        3918 non-null float64
total_household_count_of_area_type    3918 non-null float64
total_parking_capacity_in_site        3918 non-null float64
mortgage loan                         3918 non-null float64
transaction_real_price                3918 non-null float64
new_date2_도심권                         3918 non-null int

In [0]:
# test = pd.read_csv('./Colab Notebooks/data/Final_test.csv',encoding= 'utf-8')

In [47]:
test.head()

,apartment_id,Seoul,Busan,transaction_date,year_of_completion,latitude,longitude,central,district,individual,...,supply_area,tallest_building_in_sites,lowest_building_in_sites,total_household_count_in_sites,total_household_count_of_area_type,total_parking_capacity_in_site,household_loan,mean_loan,mortgage loan,transaction_real_price
0,3751,0,1,31,1984,37.519926,127.052515,0,1,0,...,0.215943,0.133333,0.202899,0.040798,0.000676,0.038399,0.622407,0.565789,0.658537,0.0
1,14029,0,1,10,1968,37.572215,126.987643,0,0,1,...,0.215894,0.133333,0.101449,0.007391,0.003378,0.010854,0.304979,0.328947,0.243902,0.0
2,12304,0,1,30,1968,37.570067,127.010640,0,0,1,...,0.048500,0.000000,0.043478,0.003104,0.019595,0.006349,0.271784,0.287281,0.235772,0.0
3,22241,0,1,10,2007,37.556938,126.859255,0,0,1,...,0.166154,0.133333,0.202899,0.047302,0.000338,0.022118,0.238589,0.282895,0.219512,0.0
4,316,0,1,10,1988,37.652256,127.082003,0,0,1,...,0.090751,0.000000,0.057971,0.016408,0.030405,0.018022,0.248963,0.285088,0.221545,0.0


In [0]:
test_original = pd.read_csv('./Colab Notebooks/data/test.csv',encoding='utf-8')

In [53]:
test_original.head()

,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,...,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price
0,462533,3751,1,200912,21~31,1984,83.58,14,37.519926,127.052515,...,15.0,district,cogeneration,6648,107.19,2,3.0,1.0,corridor,0
1,764018,14029,1,201304,1~10,1968,107.17,10,37.572215,126.987643,...,8.0,individual,gas,46184,107.17,10,0.0,0.0,corridor,0
2,813528,12304,1,201309,21~30,1968,36.17,6,37.570067,127.010640,...,4.0,individual,gas,45540,39.66,58,2.0,1.0,stairway,0
3,845097,22241,1,201312,1~10,2007,45.67,8,37.556938,126.859255,...,15.0,individual,gas,19069,87.11,1,3.0,2.0,stairway,0
4,856338,316,1,201401,1~10,1988,41.85,4,37.652256,127.082003,...,5.0,individual,gas,704,56.70,90,2.0,1.0,stairway,0


In [0]:
test['new_date'] = test_original['transaction_date']

In [55]:
test.head()

,apartment_id,Seoul,Busan,transaction_date,year_of_completion,latitude,longitude,central,district,individual,...,tallest_building_in_sites,lowest_building_in_sites,total_household_count_in_sites,total_household_count_of_area_type,total_parking_capacity_in_site,household_loan,mean_loan,mortgage loan,transaction_real_price,new_date
0,3751,0,1,31,1984,37.519926,127.052515,0,1,0,...,0.133333,0.202899,0.040798,0.000676,0.038399,0.622407,0.565789,0.658537,0.0,21~31
1,14029,0,1,10,1968,37.572215,126.987643,0,0,1,...,0.133333,0.101449,0.007391,0.003378,0.010854,0.304979,0.328947,0.243902,0.0,1~10
2,12304,0,1,30,1968,37.570067,127.010640,0,0,1,...,0.000000,0.043478,0.003104,0.019595,0.006349,0.271784,0.287281,0.235772,0.0,21~30
3,22241,0,1,10,2007,37.556938,126.859255,0,0,1,...,0.133333,0.202899,0.047302,0.000338,0.022118,0.238589,0.282895,0.219512,0.0,1~10
4,316,0,1,10,1988,37.652256,127.082003,0,0,1,...,0.000000,0.057971,0.016408,0.030405,0.018022,0.248963,0.285088,0.221545,0.0,1~10


In [56]:
# set(test.new_date)

{'11~20', '1~10', '21~28', '21~30', '21~31'}

In [0]:
list = [] 
for i in range(len(test.new_date)):
    if test.new_date[i] == '1~10':
        list.append('월초')
    elif test.new_date[i] =='11~20':
        list.append('월중')
    else:
        list.append('월말')
        

In [0]:
# set(test.year_of_completion)
list2 = []
for i in range(len(test.year_of_completion)): 
    if test.year_of_completion[i] < 1979:
        list2.append('1세대')
    elif test.year_of_completion[i] < 1990:
        list2.append('2세대')
    elif test.year_of_completion[i] < 2000:
        list2.append('3세대')
    elif test.year_of_completion[i] < 2010:
        list2.append('4세대')
    else:
        list2.append('5세대')
        

In [0]:
# set(test.transaction_month)
list3 = []
for i in range(len(test.transaction_month)):
    if test.transaction_month[i] < 4 and test.transaction_month[i] >=1:
        list3.append('q1')
    elif test.transaction_month[i] < 7 and test.transaction_month[i] >=4:
        list3.append('q2')
    elif test.transaction_month[i] < 10 and test.transaction_month[i] >=7:
        list3.append('q3')
    else:
        list3.append('q4')

In [65]:
# test['citycode'] = test.city1.astype(str).str.cat(test.city2.astype(str))
# train2.transaction_year.astype(str) # 거래년도 
# test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3918 entries, 0 to 3917
Data columns (total 39 columns):
apartment_id                          3918 non-null int64
Seoul                                 3918 non-null int64
Busan                                 3918 non-null int64
transaction_date                      3918 non-null int64
year_of_completion                    3918 non-null int64
latitude                              3918 non-null float64
longitude                             3918 non-null float64
central                               3918 non-null int64
district                              3918 non-null int64
individual                            3918 non-null int64
cogeneration                          3918 non-null int64
gas                                   3918 non-null int64
corridor                              3918 non-null int64
mixed                                 3918 non-null int64
stairway                              3918 non-null int64
city1              

In [0]:
list4 = []
s_center = ['11110','11140','11170']
s_eastnorth = ['11200','11215','11230','11260','11290','11305','11320','11350'] 
s_westnorth = ['11380','11410','11440'] 
s_westsouth = ['11470','11500','11530','11545','11560','11590','11620']
s_eastsouth = ['11650','11680','11710','11740']
p_center = ['26110','26140','26170','26200','26230','26290','26470','26500']
p_east = ['26350','26410','26260','26710']
p_west=['26320','26440','26530','26380']

for i in range(len(test.citycode)):
    if test.citycode[i] in s_center:
        list4.append('도심권')
    elif test.citycode[i] in s_eastnorth:
        list4.append('동북권')
    elif test.citycode[i] in s_westnorth:
        list4.append('서북권')
    elif test.citycode[i] in s_westsouth:
        list4.append('서남권')
    elif test.citycode[i] in s_eastsouth:
        list4.append('동남권')
    elif test.citycode[i] in p_center:
        list4.append('중부산권')
    elif test.citycode[i] in p_east:
        list4.append('동부산권')
    else:
        list4.append('서부산권')

In [0]:
test['district_seoul_pusan'] = list4 
test['transaction_quarter'] = list3
test['APT_generation'] = list2
test['new_date2'] = list

In [0]:
test['transaction'] = test.transaction_year.astype(str).str.cat(test.transaction_quarter)

In [0]:
# test.info()
test = test.drop(columns = ['transaction_date','year_of_completion','Seoul','Busan','latitude','longitude','city1','city2','city3','city4','household_loan','mean_loan','room_id','central','district','individual','cogeneration','gas','corridor','mixed','stairway','transaction_year','transaction_month','new_date','citycode','transaction_quarter'])
# train2.info()

In [71]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3918 entries, 0 to 3917
Data columns (total 18 columns):
apartment_id                          3918 non-null int64
apartment_building_count_in_sites     3918 non-null float64
bathroom_count                        3918 non-null float64
exclusive_use_area                    3918 non-null float64
floor                                 3918 non-null float64
room_count                            3918 non-null float64
supply_area                           3918 non-null float64
tallest_building_in_sites             3918 non-null float64
lowest_building_in_sites              3918 non-null float64
total_household_count_in_sites        3918 non-null float64
total_household_count_of_area_type    3918 non-null float64
total_parking_capacity_in_site        3918 non-null float64
mortgage loan                         3918 non-null float64
transaction_real_price                3918 non-null float64
district_seoul_pusan                  3918 non-null obj

In [0]:
test = pd.get_dummies(test, prefix = ['new_date2','APT_generation','district_seoul_pusan','transaction'])


In [73]:
test.head()

,apartment_id,apartment_building_count_in_sites,bathroom_count,exclusive_use_area,floor,room_count,supply_area,tallest_building_in_sites,lowest_building_in_sites,total_household_count_in_sites,...,transaction_2016q1,transaction_2016q2,transaction_2017q1,transaction_2017q2,transaction_2017q3,transaction_2017q4,transaction_2018q1,transaction_2018q2,transaction_2018q3,transaction_2018q4
0,3751,0.02439,0.2,0.240770,0.214286,0.375,0.215943,0.133333,0.202899,0.040798,...,0,0,0,0,0,0,0,0,0,0
1,14029,0.00000,0.0,0.325311,0.166667,0.000,0.215894,0.133333,0.101449,0.007391,...,0,0,0,0,0,0,0,0,0,0
2,12304,0.02439,0.2,0.070865,0.119048,0.250,0.048500,0.000000,0.043478,0.003104,...,0,0,0,0,0,0,0,0,0,0
3,22241,0.00000,0.4,0.104910,0.142857,0.375,0.166154,0.133333,0.202899,0.047302,...,0,0,0,0,0,0,0,0,0,0
4,316,0.00813,0.2,0.091221,0.095238,0.250,0.090751,0.000000,0.057971,0.016408,...,0,0,0,0,0,0,0,0,0,0


In [77]:
train3.columns

Index(['apartment_id', 'apartment_building_count_in_sites', 'bathroom_count',
       'exclusive_use_area', 'floor', 'room_count', 'supply_area',
       'tallest_building_in_sites', 'lowest_building_in_sites',
       'total_household_count_in_sites', 'total_household_count_of_area_type',
       'total_parking_capacity_in_site', 'mortgage loan',
       'transaction_real_price', 'new_date2_도심권', 'new_date2_동남권',
       'new_date2_동부산권', 'new_date2_동북권', 'new_date2_서남권', 'new_date2_서부산권',
       'new_date2_서북권', 'new_date2_중부산권', 'APT_generation_1세대',
       'APT_generation_2세대', 'APT_generation_3세대', 'APT_generation_4세대',
       'APT_generation_5세대', 'district_seoul_pusan_월말',
       'district_seoul_pusan_월중', 'district_seoul_pusan_월초',
       'transaction_2006q1', 'transaction_2006q2', 'transaction_2006q3',
       'transaction_2006q4', 'transaction_2007q1', 'transaction_2007q2',
       'transaction_2007q3', 'transaction_2007q4', 'transaction_2008q1',
       'transaction_2008q2', 'transac

In [0]:
train_data = train3[['apartment_building_count_in_sites', 'bathroom_count',
       'exclusive_use_area', 'floor', 'room_count', 'supply_area',
       'tallest_building_in_sites', 'lowest_building_in_sites',
       'total_household_count_in_sites', 'total_household_count_of_area_type',
       'total_parking_capacity_in_site', 'mortgage loan',
       'new_date2_도심권', 'new_date2_동남권',
       'new_date2_동부산권', 'new_date2_동북권', 'new_date2_서남권', 'new_date2_서부산권',
       'new_date2_서북권', 'new_date2_중부산권', 'APT_generation_1세대',
       'APT_generation_2세대', 'APT_generation_3세대', 'APT_generation_4세대',
       'APT_generation_5세대', 'district_seoul_pusan_월말',
       'district_seoul_pusan_월중', 'district_seoul_pusan_월초',
       'transaction_2006q1', 'transaction_2006q2', 'transaction_2006q3',
       'transaction_2006q4', 'transaction_2007q1', 'transaction_2007q2',
       'transaction_2007q3', 'transaction_2007q4', 'transaction_2008q1',
       'transaction_2008q2', 'transaction_2008q3', 'transaction_2008q4',
       'transaction_2009q1', 'transaction_2009q2', 'transaction_2009q3',
       'transaction_2009q4', 'transaction_2010q1', 'transaction_2010q2',
       'transaction_2010q3', 'transaction_2010q4', 'transaction_2011q1',
       'transaction_2011q2', 'transaction_2011q3', 'transaction_2011q4',
       'transaction_2012q1', 'transaction_2012q2', 'transaction_2012q3',
       'transaction_2012q4', 'transaction_2013q1', 'transaction_2013q2',
       'transaction_2013q3', 'transaction_2013q4', 'transaction_2014q1',
       'transaction_2014q2', 'transaction_2014q3', 'transaction_2014q4',
       'transaction_2015q1', 'transaction_2015q2', 'transaction_2015q3',
       'transaction_2015q4', 'transaction_2016q1', 'transaction_2016q2',
       'transaction_2016q3', 'transaction_2016q4', 'transaction_2017q1',
       'transaction_2017q2', 'transaction_2017q3', 'transaction_2017q4',
       'transaction_2018q1', 'transaction_2018q2', 'transaction_2018q3',
       'transaction_2018q4']]
train_targets = train3[['transaction_real_price']]

In [83]:
# train_data.head()
# train_targets.info()
train_data.shape


(1601458, 80)

In [90]:
test.columns 
# test.shape
# type(test.transaction_2009q4[0])

Index(['apartment_id', 'apartment_building_count_in_sites', 'bathroom_count',
       'exclusive_use_area', 'floor', 'room_count', 'supply_area',
       'tallest_building_in_sites', 'lowest_building_in_sites',
       'total_household_count_in_sites', 'total_household_count_of_area_type',
       'total_parking_capacity_in_site', 'mortgage loan',
       'transaction_real_price', 'new_date2_도심권', 'new_date2_동남권',
       'new_date2_동부산권', 'new_date2_동북권', 'new_date2_서남권', 'new_date2_서부산권',
       'new_date2_서북권', 'new_date2_중부산권', 'APT_generation_1세대',
       'APT_generation_2세대', 'APT_generation_3세대', 'APT_generation_4세대',
       'APT_generation_5세대', 'district_seoul_pusan_월말',
       'district_seoul_pusan_월중', 'district_seoul_pusan_월초',
       'transaction_2009q4', 'transaction_2013q2', 'transaction_2013q3',
       'transaction_2013q4', 'transaction_2014q1', 'transaction_2014q4',
       'transaction_2015q1', 'transaction_2015q4', 'transaction_2016q1',
       'transaction_2016q2', 'transac

In [91]:
len(['transaction_2009q4', 'transaction_2013q2', 'transaction_2013q3',
       'transaction_2013q4', 'transaction_2014q1', 'transaction_2014q4',
       'transaction_2015q1', 'transaction_2015q4', 'transaction_2016q1',
       'transaction_2016q2', 'transaction_2017q1', 'transaction_2017q2',
       'transaction_2017q3', 'transaction_2017q4', 'transaction_2018q1',
       'transaction_2018q2', 'transaction_2018q3', 'transaction_2018q4'])

18

In [92]:
len(['transaction_2006q1', 'transaction_2006q2', 'transaction_2006q3',
       'transaction_2006q4', 'transaction_2007q1', 'transaction_2007q2',
       'transaction_2007q3', 'transaction_2007q4', 'transaction_2008q1',
       'transaction_2008q2', 'transaction_2008q3', 'transaction_2008q4',
       'transaction_2009q1', 'transaction_2009q2', 'transaction_2009q3',
       'transaction_2009q4', 'transaction_2010q1', 'transaction_2010q2',
       'transaction_2010q3', 'transaction_2010q4', 'transaction_2011q1',
       'transaction_2011q2', 'transaction_2011q3', 'transaction_2011q4',
       'transaction_2012q1', 'transaction_2012q2', 'transaction_2012q3',
       'transaction_2012q4', 'transaction_2013q1', 'transaction_2013q2',
       'transaction_2013q3', 'transaction_2013q4', 'transaction_2014q1',
       'transaction_2014q2', 'transaction_2014q3', 'transaction_2014q4',
       'transaction_2015q1', 'transaction_2015q2', 'transaction_2015q3',
       'transaction_2015q4', 'transaction_2016q1', 'transaction_2016q2',
       'transaction_2016q3', 'transaction_2016q4', 'transaction_2017q1',
       'transaction_2017q2', 'transaction_2017q3', 'transaction_2017q4',
       'transaction_2018q1', 'transaction_2018q2', 'transaction_2018q3',
       'transaction_2018q4'])

52

In [93]:
len(['transaction_2006q1', 'transaction_2006q2', 'transaction_2006q3',
       'transaction_2006q4', 'transaction_2007q1', 'transaction_2007q2',
       'transaction_2007q3', 'transaction_2007q4', 'transaction_2008q1',
       'transaction_2008q2', 'transaction_2008q3', 'transaction_2008q4',
       'transaction_2009q1', 'transaction_2009q2', 'transaction_2009q3','transaction_2010q1', 'transaction_2010q2',
       'transaction_2010q3', 'transaction_2010q4', 'transaction_2011q1',
       'transaction_2011q2', 'transaction_2011q3', 'transaction_2011q4',
       'transaction_2012q1', 'transaction_2012q2', 'transaction_2012q3',
       'transaction_2012q4', 'transaction_2013q1','transaction_2014q2', 'transaction_2014q3','transaction_2015q2', 'transaction_2015q3','transaction_2016q3', 'transaction_2016q4'])

34

In [105]:
c = np.zeros((3918,34), dtype='uint8')
len(c)

3918

In [0]:
c
name = ['transaction_2006q1', 'transaction_2006q2', 'transaction_2006q3',
       'transaction_2006q4', 'transaction_2007q1', 'transaction_2007q2',
       'transaction_2007q3', 'transaction_2007q4', 'transaction_2008q1',
       'transaction_2008q2', 'transaction_2008q3', 'transaction_2008q4',
       'transaction_2009q1', 'transaction_2009q2', 'transaction_2009q3','transaction_2010q1', 'transaction_2010q2',
       'transaction_2010q3', 'transaction_2010q4', 'transaction_2011q1',
       'transaction_2011q2', 'transaction_2011q3', 'transaction_2011q4',
       'transaction_2012q1', 'transaction_2012q2', 'transaction_2012q3',
       'transaction_2012q4', 'transaction_2013q1','transaction_2014q2', 'transaction_2014q3','transaction_2015q2', 'transaction_2015q3','transaction_2016q3', 'transaction_2016q4']

addcol=pd.DataFrame(c,columns= name)

In [0]:
addcol
test2 = pd.concat([test,addcol], axis = 1)

In [113]:
test2.transaction_real_price

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
       ... 
3888    0.0
3889    0.0
3890    0.0
3891    0.0
3892    0.0
3893    0.0
3894    0.0
3895    0.0
3896    0.0
3897    0.0
3898    0.0
3899    0.0
3900    0.0
3901    0.0
3902    0.0
3903    0.0
3904    0.0
3905    0.0
3906    0.0
3907    0.0
3908    0.0
3909    0.0
3910    0.0
3911    0.0
3912    0.0
3913    0.0
3914    0.0
3915    0.0
3916    0.0
3917    0.0
Name: transaction_real_price, Length: 3918, dtype: float64

In [0]:
test_data = test2[['apartment_building_count_in_sites', 'bathroom_count',
       'exclusive_use_area', 'floor', 'room_count', 'supply_area',
       'tallest_building_in_sites', 'lowest_building_in_sites',
       'total_household_count_in_sites', 'total_household_count_of_area_type',
       'total_parking_capacity_in_site', 'mortgage loan',
       'new_date2_도심권', 'new_date2_동남권',
       'new_date2_동부산권', 'new_date2_동북권', 'new_date2_서남권', 'new_date2_서부산권',
       'new_date2_서북권', 'new_date2_중부산권', 'APT_generation_1세대',
       'APT_generation_2세대', 'APT_generation_3세대', 'APT_generation_4세대',
       'APT_generation_5세대', 'district_seoul_pusan_월말',
       'district_seoul_pusan_월중', 'district_seoul_pusan_월초',
       'transaction_2006q1', 'transaction_2006q2', 'transaction_2006q3',
       'transaction_2006q4', 'transaction_2007q1', 'transaction_2007q2',
       'transaction_2007q3', 'transaction_2007q4', 'transaction_2008q1',
       'transaction_2008q2', 'transaction_2008q3', 'transaction_2008q4',
       'transaction_2009q1', 'transaction_2009q2', 'transaction_2009q3',
       'transaction_2009q4', 'transaction_2010q1', 'transaction_2010q2',
       'transaction_2010q3', 'transaction_2010q4', 'transaction_2011q1',
       'transaction_2011q2', 'transaction_2011q3', 'transaction_2011q4',
       'transaction_2012q1', 'transaction_2012q2', 'transaction_2012q3',
       'transaction_2012q4', 'transaction_2013q1', 'transaction_2013q2',
       'transaction_2013q3', 'transaction_2013q4', 'transaction_2014q1',
       'transaction_2014q2', 'transaction_2014q3', 'transaction_2014q4',
       'transaction_2015q1', 'transaction_2015q2', 'transaction_2015q3',
       'transaction_2015q4', 'transaction_2016q1', 'transaction_2016q2',
       'transaction_2016q3', 'transaction_2016q4', 'transaction_2017q1',
       'transaction_2017q2', 'transaction_2017q3', 'transaction_2017q4',
       'transaction_2018q1', 'transaction_2018q2', 'transaction_2018q3',
       'transaction_2018q4']]
test_targets = test2[['transaction_real_price']]

In [0]:
# !pip install -q keras

In [123]:
from keras import models
from keras import layers 
from keras import backend as K

# def root_mean_squared_error(y_true, y_pred):
#         return K.sqrt(K.mean(K.square(y_pred - y_true)))

# def build_model():
    # 동일한 모델을 여러 번 생성할 것이므로 함수를 만들어 사용합니다
model = models.Sequential()
model.add(layers.Dense(128, activation='relu',
                       input_shape=(train_data.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1))
#     return model
  
# num_epochs = 100
# model = build_model()

model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

model.fit(train_data, train_targets,
          epochs=100, batch_size=500, verbose=1)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

Epoch 1/100
1601458/1601458 [==============================] - 17s 11us/step - loss: 265169664085017920.0000 - mean_absolute_error: 401558776.5718
Epoch 2/100
1601458/1601458 [==============================] - 15s 9us/step - loss: 258613050441065824.0000 - mean_absolute_error: 393511073.0089
Epoch 3/100
1601458/1601458 [==============================] - 15s 9us/step - loss: 236906822021678048.0000 - mean_absolute_error: 365579884.1980
Epoch 4/100
1601458/1601458 [==============================] - 15s 10us/step - loss: 195353464727920096.0000 - mean_absolute_error: 308122364.7913
Epoch 5/100
1601458/1601458 [==============================] - 16s 10us/step - loss: 140828785895916096.0000 - mean_absolute_error: 234503234.2747
Epoch 6/100
1601458/1601458 [==============================] - 15s 10us/step - loss: 98520811658407952.0000 - mean_absolute_error: 197721261.8306
Epoch 7/100
1601458/1601458 [==============================] - 15s 9us/step - loss: 86016624786454736.0000 - mean_absolut

KeyboardInterrupt: ignored

In [0]:
26389184971935892